# Lab 3: TableGPT. Florian MOREL and Simon ZARKA-ARTHUS


In this lab, we'll discover the power of code generation models through TableGPT2. The aim is to see how the model can be used in data analysis.

First of all, the notebook is divided into X sections:
0. Installation: This section is dedicated to module installation, model loading and data loading.
1. Guided introduction: Together, we'll discover how to use and evaluate TableGPT2.
2. More questions: You'll need to add at least one new question type to our simple evaluation system.
3. More data sets: You'll need to implement a question with multiple datasets.


IMPORTANT:
- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...
- <font color='red'>All the things you need to do are indicated in red.</font>


<font color='red'>**FIRST QUESTION:** What are the specificty of the TableGPT2 model?</font> https://huggingface.co/tablegpt/TableGPT2-7B

## 0. Setup

In [1]:
!pip install transformers datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

import pandas as pd
import torch

In [3]:
llm_name = "tablegpt/TableGPT2-7B"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
)
# Set LLM on eval mode.
llm.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-0

In [4]:
generation_config = GenerationConfig(
  max_new_tokens=512,
  do_sample=False,
  # do_sample=True,
  # temperature=.7,
  # top_p=.8,
  # top_k=20,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

In [5]:
df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
df = df.drop("Cabin", axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


## 1.1 Guided Introduction: The Model.

Below there is an example of a prompt that could be used with TableGPT2.

```
Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S
*/

Question: How many child survive? (under 18)
```

The prompt is divided in 3 parts:
1. The global instruction wich is to write python that could answer a question on a specific dataset.
2. The header of the given dataset: 5 first lines of titanic dataset.
3. The question to answer: "How many child survive? (under 18)


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `generate_answer` function following the comments inside.</font>


In [6]:
example_prompt_template = """Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

def generate_answer(prompt, llm=llm, generation_config=generation_config):

  # Create turns with the given prompt.
  messages = [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt},
  ]

  # Apply template with the tokenizer. Be careful to return pt tensors on the same device than `llm`.

  text = tokenizer.apply_chat_template(
      messages, tokenize=False, add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)

  # Generate with llm using the given generation config.
  generated_ids = llm.generate(
      **model_inputs,
      max_new_tokens=512,
      generation_config=generation_config
  )
  generated_ids = [
      output_ids[len(input_ids) :]
      for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  # Decode and select the answer to return.

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response


prompt = example_prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question="How many child survive? (under 18)",
)

answer = generate_answer(prompt)

print(prompt)
print("\n*****\n")
print(answer)

Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Hen

## 1.2 Guided Introduction: The Answer.

As you can see, the model answer with some generated code.

```
Python code:
```python
# Filter the dataframe to include only passengers under the age of 18
children = df[df['Age'] < 18]

# Count the number of children who survived
child_survivors = children[children['Survived'] == 1]

# Save the answer in the variable output
output = len(child_survivors)
```

So we will need to execute it, but there is some difficulty:
1. Sometime, the llm answer with \`\`\`python ... \`\`\`, sometime the llm answer directly with the code. We need to handle both cases.
2. We need to recover the variable output from the execution.
3. We need to evaluate single value and list of values.


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `exec_answer` function following the comments inside.</font>


In [7]:
import re

def exec_answer(answer: str, gold):

  # Extract the code from the answer. Be careful, the code is now always in ``` ```.
  reg = r"```python\n(.*)```"
  if reg_match := re.match(reg, answer, re.DOTALL):
    answer = reg_match.group(1)

  # Execute the code, https://docs.python.org/3/library/functions.html#exec
  # if the code work: Return True or False based on output == gold (be careful to handle iterable !)
  # if the code don't work return False.

  try:
    variables = {"df": df}
    exec(answer, {}, variables)
    return str(variables.get("output")) == str(gold)
  except:
    return False

print(exec_answer(answer, 61))

61
True


## 1.3 Guided Introduction: The Question.

Now we want to automatically generate questions to evaluate the performance of our model. There are benchmarks on this subject, but here we want to practice code by generating the questions ourselves.

We will generate some basic filter questions.

<font color='red'>TODO: Fill in the `generate_filter_question` function following the comments inside.</font>


In [8]:
import random

def generate_random_question(generate_function, df, k=1, seed=42):
  random.seed(seed)
  return [generate_function(df) for _ in range(k)]

def generate_filter_question(df):

  # Create a question template that take a target colunm, a filter column and a filter value

  question_template = "How many unique values are there in {target_col} for rows filtered by {filter_col} equal to {filter_val}'?"

  # Get a random target column and a random filter column (be careful they should be differnts)

  cols = random.sample(df.columns.to_list(), 2)
  target_col = cols[0]
  filter_col = cols[1]

  # Get a random filter value inside the filer column. Avoid NaN values.

  filter_val = random.sample(df[~df[filter_col].isna()][filter_col].unique().tolist(), 1)[0]

  # Compute the correct answer for the given target column, filter column and filter value.

  correct_answer = df[df[filter_col]==filter_val][target_col].unique().shape[0]

  # return formated question and associated answer in a dict {"question":[question], "answer":[answer]}

  question = question_template.format(target_col=target_col, filter_col=filter_col, filter_val=filter_val)

  return {"question": question, "answer": correct_answer}

generate_random_question(generate_filter_question, df, k=5)

[{'question': "How many unique values are there in Embarked for rows filtered by Survived equal to 0'?",
  'answer': 3},
 {'question': "How many unique values are there in Sex for rows filtered by Name equal to Stankovic, Mr. Ivan'?",
  'answer': 1},
 {'question': "How many unique values are there in Pclass for rows filtered by Survived equal to 0'?",
  'answer': 3},
 {'question': "How many unique values are there in Fare for rows filtered by SibSp equal to 1'?",
  'answer': 94},
 {'question': "How many unique values are there in PassengerId for rows filtered by Survived equal to 0'?",
  'answer': 424}]

## 1.4 Guided Introduction: The Evaluation.

The last step in this section is to evaluate our model on 20 random questions! We'll use simple accuracy.

You should have an accuracy between 0.9 and 1.

<font color='red'>TODO: Follow instruction in comment of the cell below.</font>

<font color='green'>BONUS: Investigate on errors and improve our prompt/parsing to solve them.</font>


In [9]:
from tqdm import tqdm

# Generate 20 random question

generated_questions = generate_random_question(generate_filter_question, df, k=20)

# Iterate over question to format prompt, generate answer and execute answer.
# Report the Accuracy

acc = []

for question in tqdm(generated_questions):

  prompt = example_prompt_template.format(
      var_name="df",
      df_info=df.head(5).to_string(index=False),
      user_question=question["question"],
  )

  answer = generate_answer(prompt)

  result = exec_answer(answer, question["answer"])

  acc.append(result)



print("Acc: ", sum(acc)/len(acc))

 25%|██▌       | 5/20 [00:34<01:47,  7.17s/it]

424


 50%|█████     | 10/20 [01:12<01:16,  7.61s/it]

173


100%|██████████| 20/20 [02:27<00:00,  7.36s/it]

1
Acc:  1.0


## 2. More Questions.

Now it's your turn to imagine a type of question ("How many ..."). Implement a function to generate new type of question. Verify that our previous code work with your new question then evaluate it.

<font color='red'>TODO: Generate **AT LEAST ONE** new type of question and report this new question accuracy.</font>


In [10]:
def generate_filter_question(df, question_template, query_template):

  # Get a random target column and a random filter column (be careful they should be differnts)

  cols = random.sample(df.columns.to_list(), 2)
  target_col = cols[0]
  filter_col = cols[1]

  # Get a random filter value inside the filer column. Avoid NaN values.

  filter_val = random.sample(df[~df[filter_col].isna()][filter_col].unique().tolist(), 1)[0]

  # Compute the correct answer for the given target column, filter column and filter value.

  query = query_template.format(target_col=target_col, filter_col=filter_col, filter_val=filter_val)
  code_query = "real_output = {query}".format(query=query)

  variables = {"df": df, "target_col": target_col, "filter_col": filter_col, "filter_val": filter_val}
  exec(code_query, {}, variables)

  correct_answer = variables.get("real_output")


  # return formated question and associated answer in a dict {"question":[question], "answer":[answer]}

  question = question_template.format(target_col=target_col, filter_col=filter_col, filter_val=filter_val)

  return {"question": question, "answer": correct_answer}

In [11]:
from functools import partial

generic_questions = [
    {
        "question": "How many unique values are there in {target_col} for rows filtered by {filter_col} equal to {filter_val}'?", # returns 1 int
        "query": "df[df[filter_col]==filter_val][target_col].unique().shape[0]"
    },
    {
        "question": "What is the most common value of {target_col} for rows where {filter_col} equals {filter_val}?", # returns 1 value
        "query": "df[df[filter_col]==filter_val][target_col].value_counts().idxmax()"
    },
    {
        "question": "What are the values in the '{target_col}' column where '{filter_col}' equals '{filter_val}'?", # returns a Series
        "query": "df[df[filter_col]==filter_val][target_col]"
    }
]

def make_simulation(generic_questions: list[dict], k=5):

  for sim in generic_questions:

    res = []

    partial_generate_filter_question = partial(
        generate_filter_question,
        question_template=sim["question"],
        query_template = sim["query"]
    )
    generated_questions = generate_random_question(partial_generate_filter_question, df, k)

    for question in tqdm(generated_questions):

      prompt = example_prompt_template.format(
          var_name="df",
          df_info=df.head(5).to_string(index=False),
          user_question=question["question"],
      )

      answer = generate_answer(prompt)

      result = exec_answer(answer, question["answer"])

      res.append(result)

    print("Acc : ", sum(res)/len(res))

make_simulation(generic_questions)

100%|██████████| 5/5 [00:37<00:00,  7.42s/it]


424
Acc :  1.0


100%|██████████| 5/5 [01:07<00:00, 13.53s/it]


Acc :  0.8


100%|██████████| 5/5 [00:24<00:00,  4.83s/it]

Acc :  0.6


Sometimes, we notice difference between the answer from the llm and the real answer of the query because the query has multiple good answers. The method of querying can differ the answer. Ex : the most commun element with target = 'Passenger ID'. All the Passenger ID have have a value_counts() equals to 1. Therefore, there are all a good answer.

## 3. More datasets.

Below we load a new dataset: "adult_income_dataset".

<font color='red'>TODO: Evaluate our questions on this new dataset. Report the accuracy. Comment Any differences.</font>

<font color='green'>BONUS: Try to find a prompt that answer this question: What is the mean salary of titanic surviror based on adult dataset.</font>

In [12]:
adult = pd.read_csv("hf://datasets/meghana/adult_income_dataset/adult.csv")
adult.info()

df = adult

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [13]:
make_simulation(generic_questions)

100%|██████████| 5/5 [00:36<00:00,  7.23s/it]


Acc :  1.0


100%|██████████| 5/5 [00:40<00:00,  8.05s/it]


Acc :  1.0


 20%|██        | 1/5 [00:07<00:29,  7.40s/it]

...

[0 0 0 ... 0 0 0]


 60%|██████    | 3/5 [00:53<00:43, 21.68s/it]

[]


100%|██████████| 5/5 [01:01<00:00, 12.39s/it]

Acc :  0.2


The accuracy is the same for the first question but better for the second one. There is not the previously explained problem because there is no ID column.

For the last question, the accuracy is 20%. The dataset is more complex and even if the generated query is the good one, its comparison can lead to a fail if there is only 1 different caractere.